In [5]:
'''!pip install Groq -q
!pip install groq -q'''

In [25]:
TEST_PATH = '../input/Raw Text/FAQ/faq.json'
TEXT_PATH = '../input/Raw Text/Legislation/output_legislation.json'

In [27]:
import json

test_set = json.load(open(TEST_PATH, 'r'))
legis_set = json.load(open(TEXT_PATH, 'r'))

In [7]:
from groq import Groq

client = Groq(api_key='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')

def groq_chat(content):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

In [36]:
# Inspirado no código do Leandro Carísio

perg_resposta_esperada = []

for i in range(len(test_set)):
  pr = test_set[i]

  pergunta = pr['question']
  resposta = pr['answer']
  #contexto = [item['text'] for item in pr['context']]
  #contexto = ', '.join(contexto)
 
  perg_resposta_esperada.append({"pergunta": pergunta, "resposta": resposta})

  if i < 5:
    print(f"{i}: {pergunta}\n{resposta}")
    print('----------------------------------------------------------------------')

0: Em quais hipóteses a Procuradoria Geral pode exercer a representação judicial e extrajudicial de servidores da UNICAMP?
A Procuradoria Geral da UNICAMP integra a Advocacia Pública e está vinculada à Procuradoria Geral do Estado, para fins de atuação uniforme e coordenada, nos termos do artigo 101 da Constituição Estadual. Além disso, é o órgão de representação jurídica da Universidade e de assessoramento jurídico da Reitoria, consoante prevê o artigo 95 do Regimento Geral da UNICAMP. Desta forma, conforme previsão regimental, a Procuradoria Geral tem a função institucional de defender os interesses da universidade, enquanto autarquia estadual, estritamente vinculada ao atendimento do interesse público. A representação judicial ou extrajudicial de servidor com vínculo funcional permanente poderá ocorrer em caráter excepcional, nas hipóteses em que o ato impugnado decorrer do exercício de suas atribuições constitucionais, legais ou regulamentares, quando o ato praticado tiver seguido 

In [53]:
QAC_generator_prompt = """You are a program that generates questions, answers to those questions, and context you used to answer those questions.
You will receive many text documents in .json format with "title" and "text". For each document, you will generate a question, an answer and the context
for this answer, that is, where in the text you can check the veracity of that answer.

Your response will be in .json format, translated to Brazillian Portuguese,


title: {title}
text: {text}
"""

In [52]:
legis_set[3]['titulo']

'DECRETO Nº 29.598, DE 02 DE FEVEREIRO DE 1989'

In [54]:
texto_teste = legis_set[3]

title = texto_teste['titulo']
text = texto_teste['texto']

prompt = QAC_generator_prompt.format(title=title, text=text)
response = groq_chat(prompt)  # groq_chat simulates sending prompt and getting a response

In [55]:
response

'Here is the generated question, answer, and context in Brazilian Portuguese:\n\n```\n{\n  "questão": "Quem é o Governador do Estado de São Paulo que assinou o Decreto nº 29.598?",\n  "resposta": "ORESTES QUÉRCIA",\n  "contexto": "ORESTES QUÉRCIA, Governador do Estado de São Paulo no uso de suas atribuições legais..."\n}\n```\n\nTranslation to English:\n\n```\n{\n  "question": "Who is the Governor of the State of São Paulo who signed Decree No. 29.598?",\n  "answer": "ORESTES QUÉRCIA",\n  "context": "ORESTES QUÉRCIA, Governor of the State of São Paulo in the exercise of his legal powers..."\n}\n```'

In [ ]:
import re
from tqdm import tqdm

faithfulness_values = []

# Use tqdm to wrap your loop, displaying progress as items are processed
for item in tqdm(legis_set, desc="Processing items", total=50):  # 'total' explicitly sets the expected count
    # question = item['pergunta']  # Access 'pergunta' field
    # context = item['contexto']   # Access 'contexto' field
    # answer = item['resposta']    # Access 'resposta' field
    title = item['titulo']
    text = item['texto']

    # Assume faithfulness is a formatted string that needs the question, answer, and context inserted
    prompt = QAC_generator_prompt.format(title=title, text=text)
    response = groq_chat(prompt)  # groq_chat simulates sending prompt and getting a response

In [ ]:
%%time
documents = []
all_titles = []

for item in test_set:
    for link in item["question_links"]:
        link_title = link.lower()
        if link_title in context_articles and link_title.lower() not in all_titles:
            documents.append({
                "title": link_title,
                "content": context_articles[link_title]
            })
            all_titles.append(link_title)